In [ ]:
# @title 1. Install & Setup
import os
import json
import time
from elevenlabs import ElevenLabs, VoiceSettings

# Create necessary directories
OUTPUT_DIR = 'output'
OUTPUT_AUDIO_ELEVEN_SUBDIR = 'audio/eleven'
full_output_eleven_path = os.path.join(OUTPUT_DIR, OUTPUT_AUDIO_ELEVEN_SUBDIR)
os.makedirs(full_output_eleven_path, exist_ok=True)

print("✅ Setup complete. Output folder: ", full_output_eleven_path)

In [ ]:
# @title 2. Configuration

# 🔑 REPLACE THIS WITH YOUR ACTUAL ELEVENLABS API KEY
ELEVEN_API_KEY = "sk_da55ff02071806fe7e5195dec222dcfb8c3354e7fdbe60b2"

INPUT_JSON_FILE = 'manifest.json'  # Using manifest.json for the 30 new speakers
AUDIO_WAV_DIR = 'audio_wav'
FOX_TEXT = "The quick brown fox jumps over the lazy dog. But what about the man who watched over them? He barely noticed."

print("✅ Configuration set.")

In [ ]:
# @title 3. ElevenLabs Transient Voice Logic (Normal v2.5)

def generate_eleven_audio_transient(user_id, reference_audio_path, original_text, fox_text):
    """
    Transient Voice Cloning Flow (Normal Multilingual v2):
    """
    client = ElevenLabs(api_key=ELEVEN_API_KEY)
    voice_name = f"3min-normal-{user_id}"
    voice_id = None
    
    try:
        # 1. Add Voice (Instant Voice Cloning)
        if not os.path.exists(reference_audio_path):
            print(f"   ❌ Reference file missing: {reference_audio_path}")
            return False
            
        with open(reference_audio_path, "rb") as f:
            voice = client.voices.ivc.create(
                name=voice_name,
                files=[f],
                description=f"Temporary clone for {user_id} (Normal)"
            )
        voice_id = voice.voice_id
        print(f"   ✅ Voice Created: {voice_id}")
        
        # settings for better quality
        settings = VoiceSettings(stability=0.5, similarity_boost=0.85, style=0.0, use_speaker_boost=True)

        # 2. Generate Original
        out_orig = os.path.join(full_output_eleven_path, f"spk-{user_id}-eleven-original.mp3")
        print(f"   🎙️ Generating Original Text...")
        audio_orig_gen = client.text_to_speech.convert(
            voice_id=voice_id,
            text=original_text,
            model_id="eleven_multilingual_v2",
            voice_settings=settings
        )
        with open(out_orig, "wb") as f:
            for chunk in audio_orig_gen:
                f.write(chunk)
        
        # 3. Generate Fox
        out_new = os.path.join(full_output_eleven_path, f"spk-{user_id}-eleven-new.mp3")
        print(f"   🎙️ Generating Fox Text...")
        audio_new_gen = client.text_to_speech.convert(
            voice_id=voice_id,
            text=fox_text,
            model_id="eleven_multilingual_v2",
            voice_settings=settings
        )
        with open(out_new, "wb") as f:
            for chunk in audio_new_gen:
                f.write(chunk)
                
        print(f"   ✅ Files saved: {os.path.basename(out_orig)}, {os.path.basename(out_new)}")
        return True
        
    except Exception as e:
        print(f"   ❌ ElevenLabs Error: {e}")
        return False
        
    finally:
        if voice_id:
            try:
                client.voices.delete(voice_id)
                print("   ✅ Voice Deleted.")
            except Exception as de:
                print(f"   ⚠️ Could not delete voice {voice_id}: {de}")

print("✅ ElevenLabs generation logic defined.")

In [ ]:
# @title 4. Run Batch Generation (30 manifestation speakers)

def run_batch_generation():
    with open(INPUT_JSON_FILE, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    samples = data['samples']
    total = len(samples)
    print(f"🚀 Starting generation for {total} speakers from manifest...")
    
    success_count = 0
    
    for i, sample in enumerate(samples):
        user_id = sample['user_id']
        text = sample['text']
        
        # Construct reference path from audio_wav
        wav_filename = os.path.basename(sample['audio_file']).replace('.webm', '.wav').replace('.m4a', '.wav')
        ref_path = os.path.join(AUDIO_WAV_DIR, wav_filename)
        
        # Check if already exists
        out_orig = os.path.join(full_output_eleven_path, f"spk-{user_id}-eleven-original.mp3")
        if os.path.exists(out_orig):
            print(f"⏭️ [{i+1}/{total}] Skipping {user_id} (Already exists)")
            success_count += 1
            continue
            
        print(f"🎙️ [{i+1}/{total}] User: {user_id}")
        success = generate_eleven_audio_transient(user_id, ref_path, text, FOX_TEXT)
        
        if success:
            success_count += 1
        else:
            print(f"   ❌ FAILED: {user_id}")
            
        time.sleep(1)
        
    print(f"\n🏆 Batch Generation Complete! Success: {success_count}/{total}")

# run_batch_generation()

In [ ]:
# @title 5. Update Metadata in speakers.ts (ElevenLabs Normal)

def update_eleven_metadata():
    ts_path = os.path.join(OUTPUT_DIR, 'speakers.ts')

    if not os.path.exists(ts_path):
        print(f"❌ {ts_path} not found.")
        return

    with open(ts_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # Extract JSON part
    import re
    match = re.search(r'export const speakers: Speaker\[\] = (\[.*\]);', content, re.DOTALL)
    if not match:
        print("❌ Could not parse JSON from speakers.ts")
        return

    json_str = match.group(1)
    try:
        speakers_data = json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"❌ JSON parse error: {e}")
        return

    print(f"📝 Updating metadata for {len(speakers_data)} speakers...")
    update_count = 0

    for speaker in speakers_data:
        user_id = speaker['id'].replace('spk-', '')

        # Check if ElevenLabs Normal files exist
        orig_filename = f"spk-{user_id}-eleven-original.mp3"
        new_filename = f"spk-{user_id}-eleven-new.mp3"

        path_orig = os.path.join(full_output_eleven_path, orig_filename)
        path_new = os.path.join(full_output_eleven_path, new_filename)

        if os.path.exists(path_orig) and os.path.exists(path_new):
            # Check if model already exists (eleven)
            has_normal = False
            for m in speaker['models']:
                if m['modelId'] == 'eleven':
                    has_normal = True
                    m.update({
                        "modelName": "ElevenLabs",
                        "clonedOriginalAudio": f"/{OUTPUT_AUDIO_ELEVEN_SUBDIR}/{orig_filename}",
                        "clonedOriginalText": speaker['originalText'],
                        "clonedNewAudio": f"/{OUTPUT_AUDIO_ELEVEN_SUBDIR}/{new_filename}",
                        "clonedNewText": FOX_TEXT
                    })
                    break

            if not has_normal:
                speaker['models'].append({
                    "modelId": "eleven",
                    "modelName": "ElevenLabs",
                    "clonedOriginalAudio": f"/{OUTPUT_AUDIO_ELEVEN_SUBDIR}/{orig_filename}",
                    "clonedOriginalText": speaker['originalText'],
                    "clonedNewAudio": f"/{OUTPUT_AUDIO_ELEVEN_SUBDIR}/{new_filename}",
                    "clonedNewText": FOX_TEXT
                })
                update_count += 1

    # Write back
    new_json = json.dumps(speakers_data, indent=2)
    new_content = f"export const speakers: Speaker[] = {new_json};\n"

    with open(ts_path, 'w', encoding='utf-8') as f:
        f.write(new_content)

    print(f"✅ Updated {update_count} speakers with ElevenLabs Normal data in {ts_path}")

# update_eleven_metadata()